### Startup imports

In [1]:
import pandas as pd

from autoqchem.molecule import molecule
from autoqchem.slurm_manager import slurm_manager

import logging
logging.basicConfig(level=logging.INFO)

### Initialize the slurm manager

Slurm manager is going to manage what jobs you have currently running, or created. It caches your information, so you can kill the notebook, turn-off your computer, go on vacation, and pick it up later. It will remember which jobs you haven't retrieved from the cluster, etc.

Note: it won't ask you for login unless you execute a method that requires remote access

In [3]:
sm=slurm_manager(user='zuranski', host='adroit.princeton.edu')

### Provide some smiles string(s)

and copy into them into ```my_smiles``` variable. It's a list of strings (a Python list can be of any size)

In [19]:
my_smiles=[
    "C(C1C(C(C(C(O1)O)O)O)O)O",
    "c1ccc(F)cc1",
]

For a list of smiles, you can also read them out from a csv file in this way, assumed file name ```smiles_strings.csv```, also assuming the strings are in the first column, and there are no headers

In [20]:
# my_smiles = pd.read_csv("smiles_strings.csv", header=None, usecols=[0])[0].tolist()
# my_smiles

### Initialize the molecules and generate conformations

We will use some ```for``` loops to loop over the smiles

In [21]:
mols = []
for smile in my_smiles:
    mols.append(molecule(smile, max_num_conformers=3))

INFO:autoqchem.molecule:Initializing molecule with canonical smiles: OCC1OC(O)C(C(C1O)O)O
INFO:autoqchem.molecule:Creating initial geometry with option 'best'.
INFO:autoqchem.molecule:Initial geometry created successfully.
INFO:autoqchem.molecule:Conformer Search generated 3 conformations of OCC1OC(O)C(C(C1O)O)O molecule
INFO:autoqchem.molecule:Initializing molecule with canonical smiles: Fc1ccccc1
INFO:autoqchem.molecule:Creating initial geometry with option 'best'.
INFO:autoqchem.molecule:Initial geometry created successfully.
INFO:autoqchem.molecule:Conformer Search generated 1 conformations of Fc1ccccc1 molecule


Draw some molecules

In [22]:
mols[0].draw(2)

### Create gaussian jobs for each conformation

In [23]:
for mol in mols:
    sm.create_jobs_for_molecule(mol, workflow_type="equilibrium")

INFO:autoqchem.gaussian_input_generator:Generating Gaussian input files for 3 conformations.
INFO:autoqchem.gaussian_input_generator:Generating Gaussian input files for 1 conformations.


Where is this all happening?? where are the input files?? 

Just like most of your applications it uses the 'AppData' directories, they will differ depending on the OS

In [24]:
sm.workdir

'C:\\Users\\AndrzejZuranski\\AppData\\Local\\autoqchem\\adroit'

In [25]:
sm.remote_dir

'/home/zuranski/gaussian'

In [26]:
sm.get_job_stats(split_by_can=True)

status,created
can,
Fc1ccccc1,1
OCC1OC(O)C(C(C1O)O)O,3


### Submit the jobs to the server 

if you are not connected, it will ask for password + Duo

In [27]:
sm.submit_jobs()

INFO:autoqchem.slurm_manager:Submitting 4 jobs.
INFO:autoqchem.slurm_manager:Creating connection to adroit.princeton.edu as zuranski
INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_7.4)




Password: ········


INFO:paramiko.transport:Authentication (keyboard-interactive) successful!


INFO:autoqchem.slurm_manager:Connected to adroit.princeton.edu as zuranski.
INFO:paramiko.transport.sftp:[chan 1] Opened sftp connection (server version 3)
INFO:autoqchem.slurm_manager:Submitted job e9162c5d891c1185cc2067d1ee73dccc, job_id: 696351.
INFO:autoqchem.slurm_manager:Submitted job 91ad2194b2b0eec764c9338dda81fdbd, job_id: 696352.
INFO:autoqchem.slurm_manager:Submitted job d38f6c69440bf07069e680b0aa1b5cb5, job_id: 696353.
INFO:autoqchem.slurm_manager:Submitted job 9c59cc44031ac7020ca56bace39f26ce, job_id: 696354.


Check the status of the jobs, locally and remotely

In [35]:
sm.get_job_stats(split_by_can=True)

status,submitted
can,
Fc1ccccc1,1
OCC1OC(O)C(C(C1O)O)O,3


In [36]:
sm.squeue(summary=False)

,JOBID,PARTITION,NAME,USER,ST,TIME,NODES,NODELIST(REASON)


### Retrieve jobs

If there are some finished jobs the log files will be checked for completion and downloaded locally.

In [37]:
sm.retrieve_jobs()

INFO:autoqchem.slurm_manager:There are 0 running/pending jobs, 4 finished jobs.
INFO:autoqchem.slurm_manager:Retrieving log files of finished jobs.
INFO:autoqchem.slurm_manager:4 jobs finished successfully (all Gaussian steps finished normally). 0 jobs failed.


### Resubmit failed jobs

In [40]:
sm.resubmit_failed_jobs()

INFO:autoqchem.slurm_manager:There are no failed jobs to resubmit.


### Upload molecules to the database

In [39]:
sm.upload_done_molecules_to_db(tag="tutorial_AMZ")

INFO:autoqchem.slurm_manager:There are 2 finished molecules ['Fc1ccccc1', 'OCC1OC(O)C(C(C1O)O)O'].
INFO:autoqchem.slurm_manager:Uploaded descriptors to DB for smiles: Fc1ccccc1, number of conformers: 1.
INFO:autoqchem.slurm_manager:Molecule OCC1OC(O)C(C(C1O)O)O has 0 / 3 duplicate conformers.
INFO:autoqchem.slurm_manager:Uploaded descriptors to DB for smiles: OCC1OC(O)C(C(C1O)O)O, number of conformers: 3.
